In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Scenario
A popular social media platform for sharing photos and videos has received complaints about fake user accounts. These fake accounts are said to have left spam comments on genuine user posts. Management has asked us to create a machine-learning model that will help the platform distinguish real versions from fake accounts. The company would then use the model to identify fake accounts so they can be deleted from the platform.
![](https://www.endnowfoundation.org/wp-content/uploads/elementor/thumbs/Detect-Fake-Profiles-on-Social-Media-p6yfct3ismgslao8tyklprwyrfd5tttfiwrd6xcjuw.jpg)


Image taken from: https://www.endnowfoundation.org/wp-content/uploads/elementor/thumbs/Detect-Fake-Profiles-on-Social-Media-p6yfct3ismgslao8tyklprwyrfd5tttfiwrd6xcjuw.jpg




# 1) Goal
The goal here is to predict whether a user account is fake or not. A problem of this nature is called a binary classification problem (binary since we have two categories). We use int numbers to specify the two categories. In the 'fake' column, a 1 represents that the account in that row is fake, while a 0 indicates a real account.

# 2) Gather Data
The data is in the social_media_train.csv file. The target vector is given by the 'fake' column. Here the modules that typically are needed for reading and exploration are imported and then read in pandas DataFrame df_train.

In [ ]:
# Import neccessary modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Note: Make sure module pdpipe is available. 
# You might need to install it via pip install pdpipe
import pdpipe as pdp
%matplotlib inline

# 3) Exploratory Data Analysis (EDA)
## Understand Data
It is necessary to familiarize ourselves with the data at the beginning so that we know later what to look for while cleaning and preparing the data.

In [ ]:
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Set max width of cell
pd.options.display.max_colwidth=300

# Suppress scientific notation
np.set_printoptions(suppress=True) 
pd.options.display.float_format = '{:.2f}'.format


# Display all columns
pd.set_option('display.max_columns', None)

# Read data
df_train = pd.read_csv("/kaggle/input/social-media-train/social_media_train.csv",index_col=[0])
display(df_train.head(),df_train.info())


Each line of df_train represents a user or user account.


In [ ]:
# Dataset Description
data_dict = pd.read_csv('/kaggle/input/fake-account-data-dict/fake_account__data_dict.csv', index_col = 'No.')
data_dict

In [ ]:
# Determine categorical and numerical features

# Numerical columns
num_cols = ['ratio_numlen_username','len_fullname','ratio_numlen_fullname',
            'len_desc','num_posts','num_followers',
              'num_following']
# Categorical columns
cat_cols = [col for col in df_train.columns.values.tolist() if col not in num_cols]
cat_cols

In [ ]:
# Check descriptive statistics
df_train[num_cols].describe()

In [ ]:
# Get an idea of target category share: fake
plt.figure(figsize=(15,6))
fake_share = df_train["fake"].value_counts()
mylabel=["Not fake(0)","fake(1)"]
colors = ['#99ff99','#ff9999']
plt.pie(fake_share,
        labels=mylabel,autopct="%1.1f%%",colors=colors,
        textprops={'fontsize': 16})
plt.axis("equal");

Target category is strongly balanced.


In [ ]:
# Check the percentage of the missing values

percent_missing = df_train.isnull().sum() * 100 / len(df_train)
missing_value_df = pd.DataFrame({'percent_missing (%)': percent_missing})
missing_value_df.sort_values('percent_missing (%)', ascending=False)


In [ ]:
# Correlation heatmap
# Colormap: Most negative correlations (dark-blue) to most positive correlation (dark red)
corr = df_train[num_cols].corr()
corr.style.background_gradient(cmap='coolwarm')

Just like linear regression, logistic regression makes a number of assumptions. For continuous data, the following are relevant:
* The features should not be strongly correlated with each other.
* There should be a linear relationship between the features and the sigmoid transformed probabilities.

As can be seen all correlation values between features relatively close to 0.

In [ ]:
# Pair plot oapif numerical values with fake account information
pp_cols = []
pp_cols = num_cols + ['fake']
ax=sns.pairplot(df_train[pp_cols], hue="fake",corner=True);
plt.style.use('fivethirtyeight')
ax.fig.suptitle("Pair Plot of Characteristics")

#### Very first finding:
* As can be seen from pair plots above, fake accounts are more spread around the mean for ratio_numlen_username, ratio of numeric characters in the account username to their length. Values of real accounts, in this case, are strongly clustered around their mean. 
* Most fake accounnts have less words of descriptions (len_desc) in their bio, as the average of description words for fake accounts is less than real accounts.
* ratio_numlen_fullname and ratio_numlen_username are correlated with eachother. Correlation matrix showes a value of +0.4085 for this correlation.
* Average number of posts and followers of fake accounts are close to zero.
 

Lets do some investigations of categorical data:

In [ ]:
# Categorical data
display(df_train.loc[:, cat_cols].head())
print('----------------------')

# Unique values
for col in cat_cols:
    unique_values = df_train.loc[:, col].unique()
    print("\nColumn name: {}\nUnique values: {}".format(col, unique_values))  

In [ ]:
# Define function label_encoding
def label_encoding(df):
    '''
    Function label_encoding() trnasforms categorical features
    represented by strings to binary features containing only
    0s and 1s which can be used for machine learning models.
    Input:
        DataFrame of features
    Output:
        New DataFrame with binary features    
    '''
    # label encoding
    dict_label_encoding = {'Yes': 1, 'No': 0}
    df.loc[:, 'profile_pic'] = df.loc[:, 'profile_pic'].replace(dict_label_encoding)
    df.loc[:, 'extern_url'] = df.loc[:, 'extern_url'].replace(dict_label_encoding)
    df.loc[:, 'private'] = df.loc[:, 'private'].replace(dict_label_encoding)

    # one-hot encoding
    onehot = pdp.OneHotEncode(["sim_name_username"], drop_first=False)
    # fit and transform on train set
    df = onehot.fit_transform(df)
    return df       

# Applay function label_encoding
df_train = label_encoding(df_train)
df_train.head()

# 4) Logistic Regression

In this notebook, we first focus on Logistic Regression which is a known algorithm for classification tasks. Please see [this article](https://realpython.com/logistic-regression-python/#:~:text=The%20logistic%20regression%20function%20%F0%9D%91%9D(%F0%9D%90%B1)%20is%20the%20sigmoid%20function,that%20the%20output%20is%200.) for more details about logistic regresion.

From regularization, we know that it can be helpful if a linear regression tries to minimize the slope values. For example, overfitting can be avoided.
In contrast to the logistic regression without regularization, the features must be standardized in the model with regularization. This is due to penalty parameters (l2 by default) in logistic regression: like linear regression, logistic regression with regularization makes the prediction dependent on the scaling of the features, where at 𝐿1 (Lasso) and 𝐿2 (Ridge) large coefficients are penalized more heavily. So, in order for the coefficients to be penalized equally, we need to standardize the coefficients.

## Logistic Regression With And Without Regularization
As with linear regression and its Ridge and Lasso versions, logistic regression also allows for regularization. Please see [this article](https://www.andreaperlato.com/theorypost/ridge-and-lasso-regression/) for more details about regularisations.

we can now set up a logistic regression model By default, sklearn's logistic regression algorithm already uses regularization with the regularization parameter C=1.0. If we assign an extremely large value to C, such as a 1 followed by 42 zeros (1e42), no regularization is performed. That's what we want to achieve here first.

The algorithm needs many attempts to solve the problem. The default 100 iterations are not enough. Therefore, we should also assign a relatively large number to max_iter. This parameter sets the maximum number of iterations the solvers need to converge. 10000 (1e4) should suffice here.

In [ ]:
# Import Logistic Regression model
from sklearn.linear_model import LogisticRegression

###################################################################
# a) Without regularisation

# Feature matrix and target vector
features_train = df_train.iloc[:, 1:]
target_train = df_train.loc[:, 'fake']

# Model instantiation
model_log = LogisticRegression(solver='lbfgs', max_iter=1e4, C=1e42, random_state=42)

# Model fitting
model_log.fit(features_train, target_train)

#####################################################################
# b) With regularisation

# Standardization to adjust the features
from sklearn.preprocessing import StandardScaler 

# Fit on training data and scale them
scaler = StandardScaler()
features_train_scaled = scaler.fit_transform(features_train)

# Model instantiation
model_reg = LogisticRegression(solver='lbfgs', max_iter=1e4, C=0.5, random_state=42)

# Model fitting
model_reg.fit(features_train_scaled, target_train)

# 5) Model Performance Evaluation 

We use test data from social_media_test.csv for initial classification performance:

In [ ]:
# Read data
df_test = pd.read_csv("/kaggle/input/social-media-test/social_media_test.csv",index_col=[0])
df_test = label_encoding(df_test)
df_test.head()

In [ ]:
# Feature matrix and target vector
features_test = df_test.drop('fake',axis=1)
target_test = df_test['fake']

 # Without regularisation
# predict target values from model
target_test_pred_log = model_log.predict(features_test)

# model evaluation
from sklearn.metrics import precision_score, recall_score
precision_log = precision_score(target_test, target_test_pred_log)
recall_log = recall_score(target_test, target_test_pred_log)

# print
print('Precision of model without regularisation: ', precision_log)
print('Recall of model without regularisation: ', recall_log)

#####################################################################
# With regularisation
# features matrix and target vector
features_test_scaled = scaler.transform(features_test)

# predict target values from model
target_test_pred_reg = model_reg.predict(features_test_scaled)

# model evaluation
precision_reg = precision_score(target_test, target_test_pred_reg)
recall_reg = recall_score(target_test, target_test_pred_reg)

# print
print('Precision of model with regularisation: ', precision_reg)
print('Recall of model with regularisation: ', recall_reg)

model_log is better at recognizing actual fake accounts as such (recall), while model_reg has more actual fake accounts among predicted fake accounts (precision). In other words, model_log's predictions can be trusted a little less than model_reg.

## ROC (Receiver Operating Characteristic)

LogisticRegression uses a threshold of 0.5 (50%) for determining the predicted categories. If we lower this threshold, there are more and more positive predictions and thus the recall increases. At the same time, however, the precision also decreases, because more and more of the predicted reference category cases are not fake at all.

**The question here is which classification threshold suits our specific problem well?**    We can investigate this issue with the help of [ROC (Receiver Operating Characteristic) curve](https://scikit-learn.org/stable/modules/model_evaluation.html#roc-metrics). ROC curves are a nice way to see how any predictive classifier like logistic regression model can distinguish between the true positives and negatives.
![](https://miro.medium.com/v2/resize:fit:720/format:webp/1*7NVPwda0vt4obd_Q01t9Rw.png)

The ROC curve does this by plotting sensitivity, the probability of predicting a real positive will be a positive, against 1-specificity, the probability of predicting a real negative will be a positive. 

![](https://miro.medium.com/v2/resize:fit:720/format:webp/1*b3ayb4TkvOdsNdDmJVQG1g.png)

The false positive rate is essentially a measure of how often a “false case” will occur — or, how often an actual negative instance will be classified as positive. 
Figure above demonstrates how some theoretical classifiers would plot on an ROC curve. The gray dotted line represents a classifier that is no better than random guessing — this will plot as a diagonal line. The purple line represents a perfect classifier — one with a true positive rate of 100% and a false positive rate of 0%. Nearly all real-world examples will fall somewhere between these two lines — not perfect, but providing more predictive power than random guessing. Typically, what we’re looking for is a classifier that maintains a high true positive rate while also having a low false positive rate — this ideal classifier would “hug” the upper left corner of Figure 1, much like the purple line in figure above. Image and text taken from: https://towardsdatascience.com/understanding-the-roc-curve-and-auc-dd4f9a192ecb

Back to our problem, how can we compare two different implemented model above? What are the false positive rates for model_log and model_reg? how the recall  and the false positive rate change with the threshold. This can be investigated via ROC curve.

The roc_curve() function from sklearn.metrics calculates the values ​​of the curve. It needs the predicted probabilities. In a classification model, we always use my_model.predict() to predict the categories. Instead, if we want to know the probability of belonging to one category or another, we should use [my_model.predict_proba()](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#:~:text=%5Bsource%5D-,Probability%20estimates.,-The%20returned%20estimates).

In [ ]:
# module import
from sklearn.metrics import roc_curve

# calculate roc curve values
def roc_curve_values(model,features,target):
    '''
    Function roc_curve_values estimates the probability
    and return roc_curve values as output.
    
    Input:
        model, feautes as dataframe, target values
    Output:
        False positive rate, recall, target_test_pred_proba
    '''
    # calculate probability
    target_test_pred_proba = model.predict_proba(features) 
    
    # calculate roc curve values
    false_positive_rate, recall, threshold = roc_curve(target,
                                                       target_test_pred_proba[:,1],
                                                       drop_intermediate=False)
    
    return false_positive_rate, recall, target_test_pred_proba    

In [ ]:
# Apply function roc_curve_values for model without regularization
false_positive_rate_log, recall_log, target_test_pred_proba_log  = roc_curve_values(model_log,features_test,target_test)

In [ ]:
# Apply function roc_curve_values for model with regularization
false_positive_rate_reg, recall_reg, target_test_pred_proba_reg  = roc_curve_values(model_reg,features_test,target_test)

In [ ]:
def roc_curve_plot(false_positive_rate,recall,label):
    '''
    Function roc_curve_plot plots ROC
    Input:
        false_positive_rate, recall, label: model type
    Output:
        ROC plot    
    '''
    plt.style.use('fivethirtyeight')
    fig,ax=plt.subplots()
    
    # Reference lines
    # Blue diagonal
    ax.plot([0, 1], ls = "--", label='random model')  
    # Grey vertical
    ax.plot([0, 0], [1, 0], c=".7", ls='--', label='ideal model') 
    # Grey horizontal
    ax.plot([1, 1], c=".7", ls='--')  
    
    # ROC curve
    ax.plot(false_positive_rate,recall, label = label)
    
    # labels
    ax.set_title("Receiver Operating Characteristic")
    ax.set_xlabel("False Positive Rate")
    ax.set_ylabel("Recall")
    ax.legend()

In [ ]:
roc_curve_plot(false_positive_rate_log,recall_log,'model_log')

The red curve is the outcome from the model without regularization. From the recall of around 42% and from a false positive rate of approx. 19%, it hardly deviates from the ideal. So the model appears generally very good as it follows the gray ideal rather than the blue rate line. But it's not perfect.

In [ ]:
roc_curve_plot(false_positive_rate_reg,recall_reg,'model_reg')

The ROC curve of the model with regularization (model_reg) deviates a little more from the ideal gray dashed line of the model without regularization (model_log). This suggests that the logistic regression model with regularization is slightly worse in accuracy than the non-regularized model.

## ROC AUC Measure

To create a quantifiable model performance measure from the visual impression we just used, use the area under the curve. Hence the name of the model quality measure: receiver operator characteristic area under the curve (ROC-AUC).

The [ROC AUC score ](https://www.analyticsvidhya.com/blog/2020/06/auc-roc-curve-machine-learning/#:~:text=The%20ROC%20AUC%20score%20tells,and%20the%20Negative%20class%20points.)tells us how efficient the model is. The higher the AUC, the better the model’s performance at distinguishing between the positive and negative classes. An AUC score of 1 means the classifier can perfectly distinguish between all the Positive and the Negative class points. An AUC value of 0 shows that the classifier predicts all Negatives as Positives and vice versa.
![](https://miro.medium.com/v2/resize:fit:828/format:webp/1*PU3_4LheadpGcpl6daO1mA.png)

Image source: https://medium.com/the-owl/evaluation-metrics-part-3-47c315e07222

In [ ]:
# Import roc_auc_score
from sklearn.metrics import roc_auc_score
print('roc_auc_score for model without regularization', roc_auc_score(target_test, target_test_pred_proba_log[:, 1]))
print('##########################################################')
print('roc_auc_score for model with regularization', roc_auc_score(target_test, target_test_pred_proba_reg[:, 1]))

Comparing the score of the model without regularization 96%, to model with regularization 77%, it confirms our visual impression above: the model with regularization follows the ideal less than the model without regularization. The regularization has thus harmed the model quality rather than helped it. Regardless of the chosen decision thresholds, we can conclude that we have overfitted our model!

# 6) Find The Best Logistic Regression Model With Grid Search and ROC-AUC

We can now combine the accumulated knowledge. We will use the ROC AUC measure to find the best logistic regression model and then use it to predict the fake status of the social media accounts.
Thanks to the GridSearchCV the optimal hyperparameters of models can be found automatically. We can now turn to the pipeline containing the steps from data processing and feature engineering to prediction.

Before we can start the grid search, we still have to define the grid of hyperparameters that is to be searched:
* Penalty: whether the regularization is like Ridge ('l2') or like LASSO ('l1')
* C: Regularization weakness - Inverse of regularization strength

Since we want to try both regularizations of ridge regression and lasso regression, we cannot use 'lbfgs' as before. Only the 'saga' setting supports both types of regularization. Please see [here](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#:~:text=solver%7B%E2%80%98lbfgs%E2%80%99%2C%20%E2%80%98liblinear%E2%80%99%2C%20%E2%80%98newton%2Dcg%E2%80%99%2C%20%E2%80%98newton%2Dcholesky%E2%80%99%2C%20%E2%80%98sag%E2%80%99%2C%20%E2%80%98saga%E2%80%99%7D%2C%20default%3D%E2%80%99lbfgs%E2%80%99) for more details.

In order for the algorithm to find a good result, it has to make a relatively large number of attempts. Therefore, max_iter should be increased from the default of 100 iterations. 10000 iterations (1e4) seem to be enough.

In [ ]:
# Import regured modules
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

# Define pipeline
pipeline_log = Pipeline([('scaler',StandardScaler()),('classifier',LogisticRegression(solver='saga',
                                                                  max_iter=1e4, 
                                                                  random_state=42))])
# Determine search space of hyperparameters
C_values = np.geomspace(start=0.001, stop=1000, num=14)
search_space_grid = [{'classifier__penalty': ['l1', 'l2'],
                      'classifier__C': C_values}]

# Apply Grid Search
model_grid = GridSearchCV(estimator=pipeline_log,
                          param_grid=search_space_grid,
                          scoring='roc_auc',
                          cv=5,
                          n_jobs=-1)

# Model fitting
model_grid.fit(features_train, target_train)

# Print best estimater and score
print(model_grid.best_estimator_)
print(model_grid.best_params_)
print(model_grid.best_score_)

**The logistic regression model that leads to this value uses a regularization like the ridge regression (penalty='l2') and regularizes a little less than the standard model (C=4.92).**

# 7) Model Evaluation With Test Data

It is best to check whether the extraordinarily good model quality according to cross-validation is also achieved with the test data.

In [ ]:
# Apply function roc_curve_values for model with regularization
false_positive_rate_grid, recall_grid, target_test_pred_proba  = roc_curve_values(model_grid,features_test,target_test)
roc_curve_plot(false_positive_rate_grid,recall_grid,'model_grid')

In [ ]:
# Calculated roc_auc_score
target_test_pred_proba = model_grid.predict_proba(features_test)
roc_auc_score(target_test, target_test_pred_proba[:, 1])

This value (ROC-AUC measure of 93.9%) is very good, but unfortunately slightly lower than the model figure of merit calculated using the validation data during the grid search. A discrepancy between model performance on test data and validation data is common.

# 8) Prediction


In [ ]:
# Read data
df_aim = pd.read_csv("/kaggle/input/social-media-aim/social_media_aim.csv",index_col=[0])
df_aim = label_encoding(df_aim)
features_aim = df_aim.copy()

# Apply Prediction 
df_aim.loc[:, 'fake_pred_proba'] = model_grid.predict_proba(features_aim)[:, 1]
df_aim.loc[:, 'fake_pred'] = model_grid.predict(features_aim)
df_aim

There are two Instagram accounts where the model is absolutely sure about the categorization. The predicted fake probability is either exactly 1.0 or 0.0.
For the other five accounts, the model is less certain. Here the management can decide for itself which threshold value it wants to choose in order to block accounts. The classification with the default threshold of 0.5 can be seen in the 'fake_pred' column. She is a good first serve.

User accounts are now categorized into real and fake based on their characteristics. We've even exceeded management's expectations because they can now decide for themselves how secure the classification must be before an account is removed.